<a href="https://colab.research.google.com/github/GGoYoungHee/RGBrand/blob/main/RGB_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
import pandas as pd

df2 = pd.read_csv('/content/drive/My Drive/')

ParserError: ignored

In [ ]:
df2.columns

NameError: ignored

In [ ]:
df2['R']=np.nan
df2['G']=np.nan
df2['B']=np.nan

In [ ]:
df2.head()

,names,sector,country,region,Img,year,rank,color count,meterial,shape,R,G,B
0,Samsung,Technology,South Korea,Asia Pacific,https://www.interbrand.com/assets/00000001562.png,2013,1,1,plain,text,NaN,NaN,NaN
1,Hyundai,Automotive,South Korea,Asia Pacific,https://www.interbrand.com/assets/00000001527.png,2013,2,1,plain,mix,NaN,NaN,NaN
2,Kia,Automotive,South Korea,Asia Pacific,https://www.interbrand.com/assets/00000001538.png,2013,3,3,gradation,mix,NaN,NaN,NaN
3,SK Telecom,Telecommunications,South Korea,Asia Pacific,https://www.interbrand.com/assets/00000001738.png,2013,4,2,plain,mix,NaN,NaN,NaN
4,POSCO,Diversified,South Korea,Asia Pacific,https://www.interbrand.com/assets/00000001725.png,2013,5,1,plain,text,NaN,NaN,NaN


In [ ]:
df2.dtypes

names           object
sector          object
country         object
region          object
Img             object
year             int64
rank             int64
color count      int64
meterial        object
shape           object
R              float64
G              float64
B              float64
dtype: object

이미지 다운로드

In [ ]:
import os

os.chdir('/content/drive/My Drive/kor')

In [ ]:
import urllib.request
for i in range(len(df2)):
  urllib.request.urlretrieve(df2['Img'][i],df2['names'][i]+'.jpg')  #기업 이름으로 이미지 저장


RGB 추출

In [ ]:
import cv2

from sklearn.cluster import KMeans
image = cv2.imread('/content/drive/My Drive/kor/Hana Bank.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합
K = 3
clt = KMeans(n_clusters=K)
clt.fit(image)
clt.cluster_centers_.astype(int)

array([[ 42,  62,  56],
       [251, 253, 253],
       [162, 167, 165]])

In [ ]:
centroid_histogram(clt)

array([0.1345   , 0.7242125, 0.1412875])

In [ ]:
image

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [ ]:

import numpy as np

def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram
    # based on the number of pixels assigned to each cluster
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()

    # return the histogram
    return hist

In [ ]:
clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0].astype('float64')  #가장 비중이 큰 색깔의 RGB
#0,0,0이면 흰색 -> 가장 비중이 큰 색이어도 추출 X

array([0.00000000e+00, 8.25117752e-12, 3.81401577e-11])

In [ ]:
from sklearn.cluster import KMeans
import cv2


for i in range(len(df2)):
  if df2['color count'][i] == 1:
    K = 2
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 :  
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([ 19.  40. 159.])
Dominant color is: rgb([-7.94670996e-11  1.50188058e+01  3.24269671e+01])
Dominant color is: rgb([  0.  88. 137.])
Dominant color is: rgb([ 45. 180.   0.])
Dominant color is: rgb([35. 31. 32.])
Dominant color is: rgb([35. 31. 32.])
Dominant color is: rgb([74. 67. 59.])
Dominant color is: rgb([ 58. 123. 200.])
Dominant color is: rgb([236.  28.  36.])
Dominant color is: rgb([255. 205.   0.])
Dominant color is: rgb([  6. 167. 226.])
Dominant color is: rgb([236.  28.  36.])
Dominant color is: rgb([ 12.  77. 162.])
Dominant color is: rgb([208.  17.  43.])
Dominant color is: rgb([ 67. 176.  73.])
Dominant color is: rgb([92. 55. 35.])
Dominant color is: rgb([21. 44. 83.])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 2:
    K = 3
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 : 
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([238. 116.  33.])
Dominant color is: rgb([119. 108.  97.])
Dominant color is: rgb([ 12.98265306  77.72269635 162.37402597])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if __name__ == '__main__':


Dominant color is: rgb([ 12.  77. 162.])
Dominant color is: rgb([227.04736044   6.40970693  19.38954939])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if __name__ == '__main__':


Dominant color is: rgb([ 12.  77. 162.])


...

Dominant color is: rgb([ 22.  65. 148.])
Dominant color is: rgb([-4.82280882e-13  5.70000000e+01  1.17000000e+02])
Dominant color is: rgb([238. 116.  33.])
Dominant color is: rgb([83. 82. 73.])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if __name__ == '__main__':


Dominant color is: rgb([ 12.  77. 162.])
Dominant color is: rgb([  2.  51. 106.])
Dominant color is: rgb([3.15569793e-12 1.32000000e+02 7.40000000e+01])
Dominant color is: rgb([31. 42. 81.])
Dominant color is: rgb([-5.97388805e-12  1.52000000e+02  1.29000000e+02])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if __name__ == '__main__':


Dominant color is: rgb([ 22.  65. 148.])
Dominant color is: rgb([88. 85. 58.])
Dominant color is: rgb([ 4.00000000e+00  2.25153229e-13 -6.88338275e-14])
Dominant color is: rgb([246. 128.  38.])
Dominant color is: rgb([237. 134. 179.])
Dominant color is: rgb([226.42696023  58.94042834  65.83676285])
Dominant color is: rgb([248.71327939 248.53355545 247.54193241])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 3:
    K = 4
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 :  
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([117.09001961   2.4490791   10.71479911])
Dominant color is: rgb([198.29339672  32.06888537  81.95493925])
Dominant color is: rgb([ 14.  65. 148.])
Dominant color is: rgb([108.70937396 107.24937562 107.62874625])
Dominant color is: rgb([1.20126131e-13 8.60000000e+01 1.53000000e+02])
Dominant color is: rgb([  9.  44. 116.])
Dominant color is: rgb([35. 25. 22.])
Dominant color is: rgb([ 98.09940928  99.01603376 102.03687764])
Dominant color is: rgb([109.87981157 110.06292059 112.27705249])
Dominant color is: rgb([254.24876941 248.75653162 248.9761454 ])
Dominant color is: rgb([226.05339145   6.46703895  19.44204576])
Dominant color is: rgb([191.29048208  13.10331685  63.87144623])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 4:
    K = 5
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 : 
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([ 29.60961796  67.35475234 128.94624652])
Dominant color is: rgb([-1.66267e-12  1.25000e+02  1.95000e+02])
Dominant color is: rgb([6.75015599e-14 1.25000000e+02 1.95000000e+02])
Dominant color is: rgb([250.99116131 180.90768475 131.90842131])
Dominant color is: rgb([-9.5567998e-13  1.2800000e+02  1.9700000e+02])
Dominant color is: rgb([  0.73829437 151.11492363 129.20792527])
Dominant color is: rgb([215.  25.  33.])
Dominant color is: rgb([227.06540212   6.5624431   19.53490137])
Dominant color is: rgb([-1.9095836e-12  1.2800000e+02  1.9700000e+02])
Dominant color is: rgb([  1.51183971 104.48332633 172.27308393])
Dominant color is: rgb([ 4.17514912e-11 -3.78861387e-11 -7.86428700e-11])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 5:
    K = 6
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 : 
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([230.79910581 210.39105812 178.78926975])
Dominant color is: rgb([121.89389972 127.23601009 121.3209679 ])
Dominant color is: rgb([84.08343846 88.18994413 97.37682465])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 6:
    K = 7
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 : 
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([238.68862629  90.10576071  71.65081241])
Dominant color is: rgb([215.13941692 235.07464759 232.45504058])
Dominant color is: rgb([254.34687954 254.13280116 253.8936865 ])


In [ ]:
for i in range(len(df2)):
  if df2['color count'][i] == 7:
    K = 8
    clt = KMeans(n_clusters=K)
    image = cv2.imread('/content/drive/My Drive/kor/'+df2['names'][i]+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    clt.fit(image)
    clt.cluster_centers_.astype(int)

    hist = centroid_histogram(clt)

    most = clt.cluster_centers_[np.where(np.argsort(-hist)==0)][0]
    if most[0] >= 5 and most[1] >= 5 and most[2] >= 5 and most[0] <= 250 and most[1] <= 250 and most[2] <= 250 : 
      #가장 많은 색깔이 RGB가 모두 250 이하이면 바로 RGB 추출
      print('Dominant color is: rgb({})'.format(most))
      df2.loc[i,'R'] = most[0].astype('int')
      df2.loc[i,'G'] = most[1].astype('int')
      df2.loc[i,'B'] = most[2].astype('int')
    else :
      most2 = clt.cluster_centers_[np.where(np.argsort(-hist)==1)][0]   
      #가장 많은 색깔이 검은색이면 그 다음으로 많은 색깔 RGB
      print('Dominant color is: rgb({})'.format(most2))
      df2.loc[i,'R'] = most2[0].astype('int')
      df2.loc[i,'G'] = most2[1].astype('int')
      df2.loc[i,'B'] = most2[2].astype('int')

Dominant color is: rgb([ 32.03481644  47.23447605 124.17539509])
Dominant color is: rgb([77. 77. 79.])
Dominant color is: rgb([118.79690949 191.01931567  74.50220751])
Dominant color is: rgb([  0.24597259 127.20455638  65.05097379])


In [ ]:
df2.head(n=15)[['names','color count','R','G','B']]

,names,color count,R,G,B
0,Samsung,1,19.0,39.0,159.0
1,Hyundai,1,0.0,15.0,32.0
2,Kia,3,117.0,2.0,10.0
3,SK Telecom,2,238.0,116.0,32.0
4,POSCO,1,0.0,88.0,137.0
5,KB Kookmin Bank,2,118.0,107.0,97.0
6,Samsung Life Insurance,2,12.0,77.0,162.0
7,Naver,1,44.0,179.0,0.0
8,Shinhan Card,5,230.0,210.0,178.0
9,LG,3,198.0,32.0,81.0


In [ ]:
df2.to_csv('kor_rgb.csv',index=False,header=True)